In [155]:
import seaborn as sns
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from spacy.lang.en import English
parser = English()
import nltk
import string
%matplotlib inline
from tqdm.notebook import tqdm
tqdm.pandas()

In [31]:
file_path = "./data/outputs/autonomous_clean.pkl"
df = pd.read_pickle(file_path)

We need to clean text. Since text is on web (HTML) format, we can use BeautifulSoup to parse it 

In [32]:
def soup(text):
    text = BeautifulSoup(text, "html5lib").get_text()
    return text
df['text'] = df['text'].apply(soup)

Create list of words in patent abstract that do not inform.

In [33]:
nltk.download('stopwords')
stop_list = [
'claims','claim', 'method', 'provide', 'provided', \
'device', 'devices','apparatus','system', 'systems', \
'apparatuses', 'embodiments', 'embodiment','examples', \
'example','inventions', 'invention', 'present', \
'includes', 'include', 'including','description', \
'user', 'body', 'power', 'person', 'persons', \
'comprising', 'comprise', 'comprises', 'configured', \
'configure','for example', 'discloses', 'disclose', \
'method', 'said', 'abstract', 'abstracts', 'disclosed', 'herein', \
'autonomous', 'vehicle', 'self-driving', 'sensor'
]

punctuation = list(string.punctuation)

#larger list containing all custom stop words as well as from NLP libraries
stop = set(list(stop_list) + list(ENGLISH_STOP_WORDS) \
           + stopwords.words('english') + punctuation)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
nlp = English()

In [157]:
def tokenizeText(text):
    #clean text using regex
    separators = ["\xa0\xa0\xa0\xa0", "\r", "\n",\
                  "\t", "'m", "'ll", '^\d+\s|\s\d+\s|\s\d+$']
    for i in separators:
        text = re.sub(i, " ", text.lower())
    # Use Spacy to parse text    
    doc = nlp(text)
    # Lemmatization
    tokens = [token.lemma_ for token in doc]
    # Remove stop words
    tokens = [tok for tok in tokens if len(tok) != 1 and tok not in stop]
    return tokens

def text_processing(corp):
    corp = tokenizeText(corp)
    return ' '.join(corp)

Apply text processing to every row in Dataframe

In [159]:
df['text'] = df['text'].progress_apply(text_processing)

  0%|          | 0/50 [00:00<?, ?it/s]

In [160]:
df['text'].sample(5)

priority date
2016-04-07    facilitate control steering accordance wheel p...
2017-11-22    control obstacle management receive sense datu...
2017-09-22    relate motion detection use radar technology i...
2013-06-21    implementation detect closure lane shift lane ...
2014-03-21    continuously record driving safety driver assi...
Name: text, dtype: object

In [161]:
#! python -m spacy download en

In [162]:
output_filepath = "./data/outputs/autonomous_tokenized.pkl"
df.to_pickle(output_filepath)